In [22]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [23]:
api_key = ""

In [24]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key)

In [25]:
poem = llm.invoke("write poem for love of samosa")

In [26]:
print(poem)

Oh samosa, you are the food of my heart,
Your crispy shell and warm filling are a taste of heaven.
I love the way you smell so good,
And the way you taste even better.

You are the perfect food for any occasion,
Whether I'm celebrating or just need a snack.
You always make me happy,
And I can't imagine my life without you.

Oh samosa, you are my one true love,
And I will always cherish you.

I love your golden brown color,
And the way you look so delicious.
I love the way you feel in my hands,
So soft and yet so satisfying.

I love the way you taste,
So flavorful and so addictive.
I love the way you make me feel,
So happy and so content.

Oh samosa, you are the best thing that has ever happened to me,
And I will love you forever.


In [27]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="faqs.csv", source_column="prompt")
data = loader.load()

In [42]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS

In [43]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)

In [44]:
vectordb = FAISS.from_documents(documents=data, embedding=embeddings, allow_dangerous_deserialization=True)

In [45]:
retriever = vectordb.as_retriever()
rdocs = retriever.get_relevant_documents("what are the prerequisites?")

In [46]:
rdocs

[Document(page_content='prompt: What are the things I need to know before starting this course?\nresponse: This course is for absolute beginners hence you do not need any specific skills other than basic familiarity with computers', metadata={'source': 'What are the things I need to know before starting this course?', 'row': 17}),
 Document(page_content='prompt: Is there any prerequisite for taking this course?\nresponse: The only prerequisite is that you need to have a functional laptop with at least 4GB ram, internet connection and a thrill to learn data analysis.', metadata={'source': 'Is there any prerequisite for taking this course?', 'row': 35}),
 Document(page_content='prompt: I don’t have a laptop, can I take this course?\nresponse: We recommend learning by doing and therefore you need to have a laptop or a PC (at least 4 GB ram).', metadata={'source': '\nI don’t have a laptop, can I take this course?', 'row': 25}),
 Document(page_content='prompt: Once purchased, is this course

In [47]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


In [48]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, input_key="query",return_source_documents=True, chain_type_kwargs={"prompt":PROMPT})

In [50]:
response = chain.invoke("What is the duration of the course?")

In [56]:
print(response)

{'query': 'What is the duration of the course?', 'result': 'You can complete all courses in 3 months if you dedicate 2-3 hours per day.', 'source_documents': [Document(page_content='prompt: What is the duration of this bootcamp? How long will it last?\nresponse: You can complete all courses in 3 months if you dedicate 2-3 hours per day.', metadata={'source': 'What is the duration of this bootcamp? How long will it last?', 'row': 8}), Document(page_content='prompt: Once purchased, is this course available for lifetime access?\nresponse: Yes', metadata={'source': 'Once purchased, is this course available for lifetime access?', 'row': 22}), Document(page_content='prompt: Does this bootcamp have lifetime access?\nresponse: Yes', metadata={'source': 'Does this bootcamp have lifetime access?', 'row': 7}), Document(page_content='prompt: I don’t have a laptop, can I take this course?\nresponse: We recommend learning by doing and therefore you need to have a laptop or a PC (at least 4 GB ram).'